In [84]:
import numpy as np
import cv2
from tqdm import tqdm
import os
from random import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import torch
import torchvision
import torch.nn as nn
import torchvision.datasets as datasets
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
import time
from torch.optim import Adam
from sklearn.metrics import precision_recall_fscore_support

In [66]:
train_dir = 'pictures'
test_dir = 'test'

In [67]:
def label_img(img):
    img_label = img.split('.')[0]
    return img_label

In [68]:
def create_train_data():
    imgs = []
    labels = []
    for img in tqdm(os.listdir(train_dir)):
        label = label_img(img)
        path = os.path.join(train_dir,img)
        img = cv2.imread(path)
        imgs.append([np.array(img)])
        labels.append(label)
    return imgs,labels

In [69]:
def create_test_data():
    imgs = []
    labels = []
    for img in tqdm(os.listdir(test_dir)):
        label = label_img(img)
        path = os.path.join(test_dir,img)
        img = cv2.imread(path)
        imgs.append([np.array(img)])
        labels.append(label)
    return imgs,labels

In [70]:
def apply_pca(x):
    pca = PCA(20)
    pca_x = [None] * len(x)
    for i in tqdm(range(len(x))):
        #apply pca
        img = x[i]
        blue= np.squeeze(img[:,:,0])
        green = np.squeeze(img[:,:,1])
        red = np.squeeze(img[:,:,2])
        red_transformed = pca.fit_transform(red)
        #Applying to Green channel and then applying inverse transform to transformed array.
        green_transformed = pca.fit_transform(green)
        #Applying to Blue channel and then applying inverse transform to transformed array.
        blue_transformed = pca.fit_transform(blue)
        img = np.stack((red_transformed, green_transformed, blue_transformed), axis = 2).tolist()
        pca_x[i] = (img)
    return pca_x

In [9]:
x, y = create_train_data()
x = np.squeeze(x)

100%|███████████████████████████████████████████████████████████████████████████| 20000/20000 [00:42<00:00, 475.96it/s]


In [10]:
x = apply_pca(x)

100%|████████████████████████████████████████████████████████████████████████████| 20000/20000 [22:56<00:00, 14.52it/s]


In [13]:
country_df = pd.DataFrame({'Country': y})
labelencoder = LabelEncoder()
country_df['Country'] = labelencoder.fit_transform(country_df['Country'])
country_df

,Country
0,0
1,0
2,0
3,0
4,0
...,...
19995,9
19996,9
19997,9
19998,9


In [12]:
df = pd.DataFrame({"X":x})
df = df.join(enc_df)

In [14]:
df.head()

Index(['X', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype='object')

In [15]:
df.to_csv('df.csv.gzip', compression='gzip')  

In [17]:
df = pd.read_csv('df.csv.gzip', compression = 'gzip')

,Unnamed: 0,X,0,1,2,3,4,5,6,7,8,9
0,0,"[[[-515.656611475906, -123.66000314054135, -61...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,"[[[1657.6059213222636, -506.26637486959527, 98...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,"[[[1699.1243830042176, -1195.6187278411367, 81...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,"[[[-242.87804479482452, 117.56884854261922, -7...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,"[[[472.3682836123981, -418.2295019853203, -143...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
df = df.drop(df.columns[[0]],axis = 1)

In [52]:
class MyDataset():
 
    def __init__(self,x,y):
        self.x_train=torch.tensor(x,dtype=torch.float64)
        self.x_train = torch.reshape((self.x_train), (len(self.x_train), 3, 300, 20))
        self.y_train=torch.tensor(y,dtype=torch.long)
    def __len__(self):
        return len(self.y_train)
   
    def __getitem__(self,idx):
        return self.x_train[idx],self.y_train[idx]


In [53]:
y = country_df["Country"]
md = MyDataset(x,y)

In [54]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [55]:
train_loader=DataLoader(
    md,
    batch_size=64, shuffle=True
)

In [56]:
class ConvNet(nn.Module):
    def __init__(self,num_classes=10):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (256,3,300,20)
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,300,20)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,300,20)
        self.relu1=nn.ReLU()
        #Shape= (256,12,300,20)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,150,10)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,150,10)
        self.relu2=nn.ReLU()
        #Shape= (256,20,150,10)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,150,10)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,150,10)
        self.relu3=nn.ReLU()
        #Shape= (256,32,150,10)
        
        
        self.fc=nn.Linear(in_features=150 * 10 * 32,out_features=num_classes)
        
        
        
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input.float())
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
            #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,32*150*10)
            
            
        output=self.fc(output)
            
        return output
            

In [57]:
model=ConvNet(num_classes=10).to(device)

In [58]:
#Optmizer and loss function
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [59]:
num_epochs=10

In [62]:
#Model training and saving best model

best_accuracy=0.0
train_count = 20000
for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        train_accuracy+=int(torch.sum(prediction==labels.data))
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy))
    
    torch.save(model.state_dict(),'best_checkpoint.model')

Epoch: 0 Train Loss: tensor(1.7977) Train Accuracy: 0.38325
Epoch: 1 Train Loss: tensor(1.5401) Train Accuracy: 0.47875
Epoch: 2 Train Loss: tensor(1.2254) Train Accuracy: 0.58875
Epoch: 3 Train Loss: tensor(0.9017) Train Accuracy: 0.69495
Epoch: 4 Train Loss: tensor(0.5695) Train Accuracy: 0.80965
Epoch: 5 Train Loss: tensor(0.3623) Train Accuracy: 0.8798
Epoch: 6 Train Loss: tensor(0.2598) Train Accuracy: 0.9149
Epoch: 7 Train Loss: tensor(0.2324) Train Accuracy: 0.9255
Epoch: 8 Train Loss: tensor(0.1708) Train Accuracy: 0.9458
Epoch: 9 Train Loss: tensor(0.1198) Train Accuracy: 0.96395


In [100]:
x_test,y_test = create_test_data()

100%|███████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 591.75it/s]


In [101]:
x_test = np.squeeze(x_test)
x_test = apply_pca(x_test)

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:18<00:00, 27.23it/s]


In [102]:
country_df = pd.DataFrame({'Country': y_test})
print(country_df["Country"].unique())
labelencoder = LabelEncoder()
country_df['Country_label'] = labelencoder.fit_transform(country_df['Country'])
y_test = country_df["Country_label"]

['ARG' 'AUS,' 'GBR' 'ISR' 'JPN' 'KEN' 'PRT' 'RUS' 'USA' 'ZAF']


In [76]:
mdt = MyDataset(x_test, y_test)

In [90]:
test_loader=DataLoader(
    mdt,
    batch_size=500, shuffle=True
)

In [104]:
test_accuracy = 0
recall = 0
precision = 0
test_count = 500
for i, (images,labels) in enumerate(test_loader):
    if torch.cuda.is_available():
        images=Variable(images.cuda())
        labels=Variable(labels.cuda())

    outputs=model(images)
    _,prediction=torch.max(outputs.data,1)
    test_accuracy+=int(torch.sum(prediction==labels.data))
    print(precision_recall_fscore_support(labels.data, prediction, average = None, labels = range(10)))
test_accuracy=test_accuracy/test_count
print("Test Accuracay: " + str(test_accuracy))

(array([0.28888889, 0.1       , 0.25396825, 0.26315789, 0.62068966,
       0.32075472, 0.35106383, 0.17857143, 0.14285714, 0.57894737]), array([0.26, 0.08, 0.32, 0.2 , 0.36, 0.34, 0.66, 0.2 , 0.18, 0.22]), array([0.27368421, 0.08888889, 0.28318584, 0.22727273, 0.4556962 ,
       0.33009709, 0.45833333, 0.18867925, 0.15929204, 0.31884058]), array([50, 50, 50, 50, 50, 50, 50, 50, 50, 50], dtype=int64))
Test Accuracay: 0.282
